In [1]:
import numpy as np;
from skimage import io;
from skimage.transform import rotate;
from scipy.signal import find_peaks;
import cv2;
import math;

In [ ]:
def compute_skew(image):
    '''
    This function is used to compute the angle needed to deskew image
    
    :param image: our plate image that we will compute skew angle on it
    
    :constant ksize:        kernel size used in median filtering
    :constant threshold1:   lower threshould used in canny edge detection
    :constant threshold2:   upper threshould used in canny edge detection
    :constant apertureSize: kernel size for cobel operator
    :constant L2gradient:   boolean to tell function to use more percise 
                            method in calculating gradient magnitude(mean square)
    :constant rho: 
    :constant theta:
    :constant threshold:
    :constant minLineLength:Minimum line length. Line shorter than that are rejected
    :constant maxLineGap:   Maximum allowed gap between points on the same line to link them
    
    
    :return angle: Thiis is the angle needed to fix deskewed image in degree
    '''
    
    #checking if the image is sutable for processing
    if len(image.shape) == 3:
        h, w, _ = image.shape
    elif len(image.shape) == 2:
        h, w = image.shape
    else:
        print('upsupported image type')
        return 0;

    #using median filter to remove unrequried noise and very weak edges
    img = cv2.medianBlur(image, ksize = 3)

    #applying canny edge detection to extract edges that exists in the plate
    edges = cv2.Canny(img,  threshold1 = 30,  threshold2 = 100, apertureSize = 3, L2gradient = True)
    
    #
    lines = cv2.HoughLinesP(edges, rho = 1, theta = math.pi/180, threshold = 30, minLineLength = w / 4.0, maxLineGap = h/4.0)
    
    angle = 0.0
    nlines = lines.size
    #calculating sum of the anles of the lines resulting from previous function 
    count = 0
    for x1, y1, x2, y2 in lines[0]:
        ang = np.arctan2(y2 - y1, x2 - x1)
        if math.fabs(ang) <= 30: # excluding line angles with extreme rotations(outliars)
            angle += ang
            count += 1

    if count == 0:
        return 0;
    
    #returns average angle in degrees
    return (angle / count)*180/math.pi

In [ ]:
def rotate_image(image, angle):
    '''
    This function is a combination of both the abovr
    
    :param image: our plate image could be RGB or Grayscale
    
    :return image: the plate image after rotation
    ''' 
    result = rotate(image, angle, cval = 0);
    return result

In [ ]:
'''
    This function is just to rotate our image
    
    :param image: our plate image could be RGB or Grayscale
    :param angle: angle provided to the function to rotate image with
    
    :constant cval: represents the value to be assigned to the out of
                    bounds pixels that will appear due to rotation
    
    :return image: the plate image after rotation
    ''' 
def deskew(image):
    return rotate_image(image, compute_skew(image))

In [ ]:
im = cv2.imread('D:\GitHub\License-Plate-Recognition\Cars33.png');
newImage = deskew(im);
io.imshow(newImage);

In [2]:
def extract_characters(img):
    
    GaussianFilter= cv2.GaussianBlur(img, (5,5), 0);
    #binary = cv2.threshold(GaussianFilter, 0, 255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    _, binary = cv2.threshold(GaussianFilter, 80, 255, cv2.THRESH_BINARY_INV);

    kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    thre_mor = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel3);
    thre_mor = thre_mor/255
    io.imshow(thre_mor);
    vertical_projection = np.sum(thre_mor, axis = 0);
    
    start = []
    end = []
    for i in range(vertical_projection.size -1):
        if(vertical_projection[i] == 0):
            if(vertical_projection[i+1] != 0):
                start.append(i)
        if(vertical_projection[i] != 0):
            if(vertical_projection[i+1] == 0):
                end.append(i)
    characters = []
    for i in range(len(start)):
        characters.append(img[:,start[i]:end[i]])
    return characters;



In [5]:
im = cv2.imread('D:\GitHub\License-Plate-Recognition\Cars14.png', cv2.IMREAD_GRAYSCALE);
extract_characters(im);

error: OpenCV(4.0.1) C:\ci\opencv-suite_1573470242804\work\modules\core\src\matrix.cpp:757: error: (-215:Assertion failed) dims <= 2 && step[0] > 0 in function 'cv::Mat::locateROI'
